In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

In [2]:
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
from scipy.ndimage.interpolation import shift

In [4]:
import os
for dirname, _, filenames in os.walk("input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

input\Dig-MNIST.csv
input\sample_submission.csv
input\test.csv
input\train.csv


In [5]:
train_data = pd.read_csv("input\\train.csv")

In [6]:
y_temp = train_data.pop("label")
train_data_x = np.asarray(train_data)
train_data_x = train_data_x.reshape(60000,28,28,1)
train_data_y = np.asarray(y_temp)

In [7]:
eval_data = pd.read_csv("input\Dig-MNIST.csv")

In [8]:
y_temp = eval_data.pop("label")
eval_data_x = np.asarray(eval_data)
eval_data_x = eval_data_x.reshape(10240,28,28,1)
eval_data_y = np.asarray(y_temp)

In [9]:
test_data = pd.read_csv("input\\test.csv")

In [10]:
test_data.pop("id")
test_data_np = np.asarray(test_data)
test_data_np = test_data_np.reshape(5000,28,28,1)

In [11]:
def augment_data(dataset, dataset_labels, augementation_factor=1, use_random_rotation=True, use_random_shear=True, use_random_shift=True):
    augmented_image = []
    augmented_image_labels = []

    for num in range (0, dataset.shape[0]):

        for i in range(0, augementation_factor):
            # original image:
            augmented_image.append(dataset[num])
            augmented_image_labels.append(dataset_labels[num])

            if use_random_rotation:
                augmented_image.append(tf.keras.preprocessing.image.random_rotation(dataset[num], 20, row_axis=0, col_axis=1, channel_axis=2))
                augmented_image_labels.append(dataset_labels[num])

            if use_random_shear:
                augmented_image.append(tf.keras.preprocessing.image.random_shear(dataset[num], 0.2, row_axis=0, col_axis=1, channel_axis=2))
                augmented_image_labels.append(dataset_labels[num])

            if use_random_shift:
                augmented_image.append(tf.keras.preprocessing.image.random_shift(dataset[num], 0.2, 0.2, row_axis=0, col_axis=1, channel_axis=2))
                augmented_image_labels.append(dataset_labels[num])
                
    return np.array(augmented_image), np.array(augmented_image_labels)

In [12]:
train_data_augmented_x,train_data_augmented_y  = augment_data(train_data_x,train_data_y,augementation_factor=1)

In [13]:
train_data_augmented_x.shape

(240000, 28, 28, 1)

In [40]:
model = models.Sequential()

In [41]:
model.add(layers.Conv2D(28, (3, 3), activation='relu', input_shape=(28,28,1)))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(38, (2, 2), activation='relu'))
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(64, (2, 2), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(108, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (2, 2), activation='relu'))

In [42]:
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [43]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 26, 26, 28)        280       
_________________________________________________________________
batch_normalization_25 (Batc (None, 26, 26, 28)        112       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 24, 24, 32)        8096      
_________________________________________________________________
batch_normalization_26 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 11, 11, 38)        4902      
_________________________________________________________________
batch_normalization_27 (Batc (None, 11, 11, 38)       

In [46]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_data_augmented_x, train_data_augmented_y, epochs=20, 
                    validation_data=(eval_data_x, eval_data_y))

Epoch 1/20
7500/7500 [==============================] - 418s 56ms/step - loss: 1.4703 - accuracy: 0.9907 - val_loss: 1.6039 - val_accuracy: 0.8569
Epoch 2/20
7500/7500 [==============================] - 410s 55ms/step - loss: 1.4710 - accuracy: 0.9901 - val_loss: 1.6203 - val_accuracy: 0.8409
Epoch 3/20
7500/7500 [==============================] - 409s 55ms/step - loss: 1.4705 - accuracy: 0.9905 - val_loss: 1.5752 - val_accuracy: 0.8856
Epoch 4/20
7500/7500 [==============================] - 409s 55ms/step - loss: 1.4702 - accuracy: 0.9908 - val_loss: 1.6295 - val_accuracy: 0.8313
Epoch 5/20
7500/7500 [==============================] - 410s 55ms/step - loss: 1.4705 - accuracy: 0.9905 - val_loss: 1.6445 - val_accuracy: 0.8166
Epoch 6/20
7500/7500 [==============================] - 410s 55ms/step - loss: 1.4713 - accuracy: 0.9897 - val_loss: 1.6223 - val_accuracy: 0.8386
Epoch 7/20
7500/7500 [==============================] - 403s 54ms/step - loss: 1.4700 - accuracy: 0.9910 - val_loss: 1

In [47]:
model.save_weights('saved_weights.hdf5', overwrite=True)

In [48]:
pred = model.evaluate(eval_data_x, eval_data_y, batch_size=128)
print("test loss, test acc:", pred)

80/80 [==============================] - 2s 25ms/step - loss: 1.5921 - accuracy: 0.8687
test loss, test acc: [1.5920648574829102, 0.8687499761581421]
